# Pandas: Union Tables

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('./../data/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('./../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### Сколько строк в файле рейтингов, не считая строки заголовка:

In [4]:
ratings.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

#### Какое количество жанров имеют фильмы в датасете movies?

Каждый фильм может относиться как к одному, так и к нескольким жанрам верно

#### Какое минимальное значение принимает выставленная оценка в датасете ratings?

In [5]:
ratings['rating'].min()

0.5

#### Какое максимальное значение принимает выставленная оценка в датасете ratings?

In [6]:
ratings['rating'].max()

5.0

## Полезные ссылки
- Наглядный [пример](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) различных режимов склейки таблиц по строкам или столбцам (метод concat). Пригодится, чтобы быстро вспомнить, как изменять типы объединения таблиц.
- [Документация метода merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) — пригодится, если забылись названия основных параметров (по каким столбцам объединяем таблицы и каким способом).
- [Объяснение](http://www.skillz.ru/dev/php/article-Obyasnenie_SQL_obedinenii_JOIN_INNER_OUTER.html) типов объединений таблиц — если возникнут трудности с выбором типа объединения (в pandas.merge параметр how).

#### Сколько фильмов в таблице movies?

In [7]:
movies.count()

movieId    9742
title      9742
genres     9742
dtype: int64

## Объединяем таблицы

In [8]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Схематично метод merge можно описать так: joined = left_df.merge(right_df, on='', how='').

Давайте разберем подробнее параметры метода: 

- **left_df** / **right_df** — датафреймы, которые мы объединяем. К "правому" датафрейму присоединяем "левый" (в нашем примере "левый" датафрейм — ratings, "правый" — movies). 
- **how** — параметр объединения записей. Он может иметь четыре значения: *left, right, inner и outer*.
  - *left* берем все записи (movieId) из "левого" датафрейма (ratings) и ищем их соответствия в "правом" (movies). В итоговом датафрейме останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.
  - *right* остаются только значения из "правого" датафрейма. Если совпадений между таблицами нет, то ставим нулевое значение.
  - *inner* оставляет только те записи (movieId), которые есть в обоих датафреймах.
  - *outer* объединяет все варианты movieId в обоих датафреймах. 
- **on** определяет, по какому столбцу происходит объединение. Для объединения по нескольким столбцам используйте on = ['col1', 'col2'] или left_on и right_on.


После объединения датафреймов лучше проверять, что не возникло дубликатов. О возможных проблемах метода merge мы поговорим в следующем блоке. Сейчас убедимся в том, что число строк объединенного датафрейма совпадает с исходным:

In [9]:
len(ratings) == len(joined)

True

Получаем значение True — значит, число строк совпадает.

## Трудности объединения датафреймов

In [10]:
ratings_small = pd.read_csv('./../data/ratings_example.txt', sep = '\t')
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [11]:
movies_small = pd.read_csv('./../data/movies_example.txt', sep = '\t')
movies_small.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


#### В текущей версии датасета movies значение movieId = 31 встречается дважды, а movieId = 32 - один раз. Определите, при каких типах объединения датафреймов ratings и movies по столбцу movieId в итоговом датафрейме останутся оба значения movieId, при условии использования следующего синтаксиса:

ratings_small.merge(movies_small, on = 'movieId', how = ...)

In [12]:
ratings_small.merge(movies_small, on='movieId', how='right')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [13]:
ratings_small.merge(movies_small, on='movieId', how='outer')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


## Pandas: Дубликаты строк

In [14]:
ratings_small.merge(movies_small, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


### Удаляем дубликаты
Если вы хотите избежать подобной ситуации, необходимо удалить дубликаты из таблицы movies. Для этого подходит метод **drop_duplicates**. В параметре subset указываем один или несколько столбцов, по комбинации которых хотим удалить дубликаты.

С помощью параметра **keep** указываем, какой из встречающихся дубликатов оставить (например, первый или последний). Параметр **inplace** указывает, что изменения нужно сохранить в датафрейме, к которому применяется метод (в нашем случае — в датафрейме movies):

In [17]:
movies_small_uniq = movies_small.drop_duplicates(subset='movieId', keep='first', inplace=False)
movies_small_uniq.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


Теперь объединение таблиц будет корректным:

In [18]:
ratings_small.merge(movies_small_uniq, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


#### При каком типе объединения таблиц с помощью метода merge (т. е. при каком значении параметра how) не могут возникать дубликаты строк? В качестве примера можете использовать объединение датафреймов ratings_small и movies_small из этого шага:

в любом типе могут быть дубликаты верно

In [23]:
ratings_small.merge(movies_small, how='outer', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


## Tasks:

В этой серии заданий мы разберемся с данными новых поступлений интернет-магазина. В словаре items_dict (который мы переведем в датафрейм) содержится информация о наличии товара на складе:

In [24]:
items_dict = {
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
}

А в словаре purchase_log — данные о покупках товаров:

In [25]:
purchase_log = {
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
}

- item_id — идентификатор модели (по этому столбцу будем объединять датафреймы)
- vendor — производитель модели
- stock_count — имеющееся на складе количество данных моделей (в штуках)
- purchase_id — идентификатор покупки
- price — стоимость модели в покупке

Переведем сначала эти словари в датафреймы для удобства работы:

In [27]:
items_df = pd.DataFrame(items_dict)
display(items_df)

,item_id,vendor,stock_count
0,417283,Samsung,54
1,849734,LG,33
2,132223,Apple,122
3,573943,Apple,18
4,19475,LG,102
5,3294095,Apple,43
6,382043,Samsung,77
7,302948,Samsung,143
8,100132,LG,60
9,312394,ZTE,19


In [28]:
purchase_df = pd.DataFrame(purchase_log)
display(purchase_df)

,purchase_id,item_id,price
0,101,417283,13900
1,101,849734,5330
2,101,132223,38200
3,112,573943,49990
4,121,19475,9890
5,145,3294095,33000
6,145,382043,67500
7,145,302948,34500
8,145,103845,89900
9,221,100132,11400


#### Объедините получившиеся датафреймы по столбцу item_id с типом outer.

#### Определите, модель с каким item_id есть в статистике продаж purchase_df, но не учтена на складе (подсказка: подумайте, какой датафрейм должен быть "левым", а какой "правым", чтобы получить необходимые данные). Введите ответ в виде целого числа.

In [36]:
purchase_df.merge(items_df, how='left', on='item_id')

,purchase_id,item_id,price,vendor,stock_count
0,101,417283,13900,Samsung,54.0
1,101,849734,5330,LG,33.0
2,101,132223,38200,Apple,122.0
3,112,573943,49990,Apple,18.0
4,121,19475,9890,LG,102.0
5,145,3294095,33000,Apple,43.0
6,145,382043,67500,Samsung,77.0
7,145,302948,34500,Samsung,143.0
8,145,103845,89900,NaN,NaN
9,221,100132,11400,LG,60.0


#### Решите обратную задачу: модель с каким item_id есть на складе, но не имела ни одной продажи? Введите ответ в виде целого числа.

In [41]:
items_df.merge(purchase_df, how='left', on='item_id')

,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101.0,13900.0
1,849734,LG,33,101.0,5330.0
2,132223,Apple,122,101.0,38200.0
3,573943,Apple,18,112.0,49990.0
4,19475,LG,102,121.0,9890.0
5,3294095,Apple,43,145.0,33000.0
6,382043,Samsung,77,145.0,67500.0
7,302948,Samsung,143,145.0,34500.0
8,100132,LG,60,221.0,11400.0
9,312394,ZTE,19,NaN,NaN


#### Сформируйте датафрейм merged, в котором в результате объединения purchase_df и items_df останутся модели, которые учтены на складе и имели продажи. Сколько всего таких моделей?

In [43]:
merged = items_df.merge(purchase_df, how='inner', on='item_id')
display(merged)

,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101,13900
1,849734,LG,33,101,5330
2,132223,Apple,122,101,38200
3,573943,Apple,18,112,49990
4,19475,LG,102,121,9890
5,3294095,Apple,43,145,33000
6,382043,Samsung,77,145,67500
7,302948,Samsung,143,145,34500
8,100132,LG,60,221,11400


In [44]:
merged.count()

item_id        9
vendor         9
stock_count    9
purchase_id    9
price          9
dtype: int64

#### Посчитайте объем выручки для каждой модели, которую можно получить, распродав все остатки на складе. Модель с каким item_id имеет максимальное значение выручки после распродажи остатков? Ответ дайте в виде целого числа.

Примечание: перемножение столбцов датафрейма можно производить разными способами, но самый простой - перемножение "в лоб" вида df['col1'] = df['col2'] * df['col3']. Для присоединения новых данных к датафрейму тоже можно использовать различные методы, включая функцию .append(), которая позволяет присоединять к датафрейму другой датафрейм, серии или словари.

In [55]:
count_price_df = merged[['item_id', 'stock_count', 'price']]
display(count_price_df)

,item_id,stock_count,price
0,417283,54,13900
1,849734,33,5330
2,132223,122,38200
3,573943,18,49990
4,19475,102,9890
5,3294095,43,33000
6,382043,77,67500
7,302948,143,34500
8,100132,60,11400


In [63]:
count_price_sum_df = count_price_df.copy()
count_price_sum_df['total_sum_of_item'] = count_price_df.apply(lambda row: row['stock_count'] * row['price'], axis=1)
display(count_price_sum_df)

,item_id,stock_count,price,total_sum_of_item
0,417283,54,13900,750600
1,849734,33,5330,175890
2,132223,122,38200,4660400
3,573943,18,49990,899820
4,19475,102,9890,1008780
5,3294095,43,33000,1419000
6,382043,77,67500,5197500
7,302948,143,34500,4933500
8,100132,60,11400,684000


In [64]:
count_price_sum_df.sort_values(['total_sum_of_item'], ascending=False).head(1)

,item_id,stock_count,price,total_sum_of_item
6,382043,77,67500,5197500


#### Посчитайте итоговую выручку из прошлого задания по всем моделям. Ответ дайте в виде целого числа.

In [65]:
count_price_sum_df['total_sum_of_item'].sum()

19729490

## Pandas: Объединяем выгрузки

Данные могут приходить к нам в самом различном виде. Часто это не один файл, а целая папка с множеством файлов и не всегда предсказуемыми/совпадающими именами. Объединить набор файлов в один датафрейм, нам поможет библиотека os.

На этом шаге мы получим список всех имен файлов в папке с данными, что позволит нам считывать данные в датафрейм, не обращая внимания на имена файлов. Для решения задачи работаем с архивной папкой с выгрузками данных (data.zip). В папке содержится набор из 10 файлов. В текущем примере это знакомый нам ratings.csv, разбитый на 10 частей.

В теории мы можем создать 10 датафреймов с 10 именами, загрузить в каждый по файлу и далее заняться их объединением. Но такой подход крайне непрактичен: что, если в следующий раз выгрузка будет состоять из 100 файлов? А может, из сотен тысяч? Давайте автоматизируем этот процесс.

Импортируем библиотеку os:

In [66]:
import os

In [67]:
files = os.listdir('./../data/ratings')
files

['ratings_1.txt',
 'ratings_10.txt',
 'ratings_2.txt',
 'ratings_3.txt',
 'ratings_4.txt',
 'ratings_5.txt',
 'ratings_6.txt',
 'ratings_7.txt',
 'ratings_8.txt',
 'ratings_9.txt']

#### После чтения папки список файлов оказался следующим:

files2 = ['setup.py', 'ratings.txt', 'stock_stats.txt', 'movies.txt', 'run.sh', 'game_of_thrones.mov']

#### Создайте на основе списка files новый список data, в который поместите только файлы, содержащие в названии "txt".

In [71]:
files2 = ['setup.py', 'ratings.txt', 'stock_stats.txt', 'movies.txt', 'run.sh', 'game_of_thrones.mov']
list(filter(lambda x: x.endswith('.txt'), files2))

['ratings.txt', 'stock_stats.txt', 'movies.txt']

## Выгрузка из вложенных папок

Если бы в папке 'ratings' содержались вложенные папки, то получить их имена отдельно от названий файлов можно было бы с помощью метода walk.

Представим, что в основной папке 'ratings' лежит подпапка 'subfolder' с файлом 'file_in_subfolder.txt'. Тогда с помощью вызова метода os.walk('ratings') для каждой вложенной папки получим три значения: имя корневой папки, список вложенных папок и названия файлов. Запишем эти значения в переменные, которые назовем root, dirs и files:

In [73]:
for root, dirs, files in os.walk('./../data/ratings'):
    print(root, dirs, files)

./../data/ratings ['subfolder'] ['ratings_1.txt', 'ratings_10.txt', 'ratings_2.txt', 'ratings_3.txt', 'ratings_4.txt', 'ratings_5.txt', 'ratings_6.txt', 'ratings_7.txt', 'ratings_8.txt', 'ratings_9.txt']
./../data/ratings\subfolder [] ['rating_11.txt']


Вторая строка data\subfolder... показывает структуру вложенной папки subfolder. Таким образом вы можете получать всю структуру папки вместе с файлами и вложенными директориями.

## Склеивание датафреймов

На прошлом шаге мы получили список файлов в папке. Теперь нам необходимо содержимое этих файлов, собрать в одном датафрейме. Это можно сделать используя метод concatenate.

Этот метод позволяем склеивать два и более датафреймов в один. Например, если у нас есть датафреймы с одинаковыми столбцами, то единую таблицу можно получить следующим образом:

In [82]:
rating_dfs = []

for root, dirs, files in os.walk('./../data/ratings'):
    for file in files:
        path = os.path.join(root, file)        
        temp = pd.read_csv(path, names=['userId', 'movieId', 'rating', 'timestamp'])        
        rating_dfs.append(temp)

total_rating_df = pd.concat(rating_dfs)

total_rating_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Кстати, датафреймы можно склеивать не только по строкам (т. е. по вертикали), но и по горизонтали (если число строк у них одинаковое). Для второго варианта необходимо использовать параметр axis = 1: total_dataframe = pd.concat([df1, df2], axis = 1).

Воспользуемся этим приёмом: при прохождении по файлам папки data будем записывать содержимое каждого файла в датафрейм temp и добавлять его к итоговому датафрейму data. Перед этим создадим пустой датафрейм data с нужными названиями столбцов:

In [83]:
data = pd.DataFrame(columns = ['userId', 'movieId', 'rating', 'timestamp'])

Для каждого имени файла filename будем записывать его содержимое в датафрейм temp:

In [84]:
for root, dirs, files in os.walk('./../data/ratings'):
    for file in files:
        path = os.path.join(root, file)        
        temp = pd.read_csv(path, names=['userId', 'movieId', 'rating', 'timestamp'])
        # И «прибавлять» содержимое очередного файла к data:
        data = pd.concat([data, temp])

data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


#### Напишите цикл, который собирает содержимое файлов папки data в единый датафрейм data.
#### Сколько строк в датафрейме data?

In [86]:
data['movieId'].count()

100004